**Keyword Extraction using KeyBert and Yake**

Reference: https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/

In [ ]:
!pip3 install keybert
!pip3 install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23794 sha256=31110dbc95891c65d1e23735ffd749aea01d7bd5a5c37e7514927d2a5cbdaa9d
  Stored in directory: /root/.cache/pip/wheels/68/aa/41/82025d89b0eb97484c9ac7d527abf596765c41733af79f86b0
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=8f1ada9d36eecccc8e55b2db686983db5d6d41d319f7f2

In [ ]:
# connect your personal google drive to store dataset and trained model
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", -1)
from keybert import KeyBERT
import yake

In [ ]:
# Define all functions

def get_source_from_excel(dataset):
  source=pd.read_excel(dataset, sheet_name='Sheet_name_1')
  return source

def keybert_extract(context):
  kw_BERT = KeyBERT()
  bert_keywords = kw_BERT.extract_keywords(context)
  bert_df = pd.DataFrame(bert_keywords, columns =['keyword', 'normalized_weight'])
  return bert_df

def yake_extract(context):
  kw_yake = yake.KeywordExtractor(n=2,top=5)
  yake_keywords = kw_yake.extract_keywords(context)
  yake_df = pd.DataFrame(yake_keywords, columns =['keyword', 'weight']).sort_values(by=['weight', 'keyword'], ascending=[True, False])
  yake_df['normalized_weight'] = 1 - ( yake_df['weight']  / sum(yake_df['weight']))
  yake_df.drop(['weight'], axis=1, inplace = True)
  return yake_df

def get_keywords_from_context(context):
  keywords_df = pd.concat([yake_extract(context), keybert_extract(context)], axis=0)    
  return keywords_df

def get_keywords_for_source(source):
  context_keywords = pd.DataFrame()
  for index, row in source.iterrows():
    context = row['content'] 
    keywords = get_keywords_from_context(context)
    c_keywords  = pd.DataFrame()
    c_keywords['context'] = [context]
    c_keywords = keywords.merge(c_keywords, how='cross')
    context_keywords = pd.concat([context_keywords , c_keywords], axis=0)
  return  context_keywords


In [ ]:
#Usage
source = get_source_from_excel("/content/gdrive/My Drive/CS 677 Project/dataset/chemistryoflife_v3.xlsx")
context_keywords = get_keywords_for_source(source)
context_keywords.to_excel("/content/gdrive/My Drive/CS 677 Project/dataset/context_keywords.xlsx", sheet_name='Sheet_name_1')  